<a href="https://colab.research.google.com/github/VishnuRaja98/Hyperbolic-Embeddings/blob/main/Hyperbolic_Embeddings_on_Exaggeration_sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
# copied and modified from https://github.com/HazyResearch/hyperbolics/blob/master/scratch/ParseTreeEmbedding.ipynb
import os
import numpy as np
import scipy.sparse.csgraph as csg
from joblib import Parallel, delayed
import multiprocessing
import networkx as nx
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math


from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

#BEGIN imports ive added
import csv
from transformers import AutoTokenizer, AutoModel
from graphviz import Digraph
#END imports ive added

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
lines = []
file_path = '/gdrive/My Drive/NLP AP/exag_data/pos_headline_pairs.tsv'
with open(file_path, 'r', newline='', encoding='utf-8') as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    for row in reader:
        lines.append(row)

print(f"Total relations = {len(lines)}")

neg_lines = []
file_path = '/gdrive/My Drive/NLP AP/exag_data/neg_headline_pairs.tsv'
with open(file_path, 'r', newline='', encoding='utf-8') as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    for row in reader:
        neg_lines.append(row)

print(f"Total negative relations of similar sentences = {len(neg_lines)}")

Total relations = 804
Total negative relations of similar sentences = 788


In [5]:
sentence_ids = {}
i=0
for line in lines:
    if line[0] not in sentence_ids:
        sentence_ids[line[0]]=i
        i=i+1
    if line[1] not in sentence_ids:
        sentence_ids[line[1]]=i
        i=i+1

for line in neg_lines:
    if line[0] not in sentence_ids:
        sentence_ids[line[0]]=i
        i=i+1
    if line[1] not in sentence_ids:
        sentence_ids[line[1]]=i
        i=i+1

print("Unique Sentences = ", sentence_ids)
unique_sentences_list = ["" for _ in range(len(sentence_ids))]
for key, val in sentence_ids.items():
    unique_sentences_list[val]=key
print("unique_sentences_list = ", unique_sentences_list)

Unique Sentences =  {'Thailand refuses entry to cruise ship with no coronavirus cases': 0, 'Thailand bans cruise ship even without any coronavirus cases on board.': 1, 'Thailand decides on cruise ship with no coronavirus cases': 2, 'Thailand bans cruise ships with no coronavirus cases': 3, 'Thailand refuses entry to cruise ship, concerned with coronavirus cases': 4, 'Thailand denies entry to all cruise ships regardless of coronavirus cases.': 5, 'Cruise ship wrongfully denied entry into Thailand despite no coronavirus cases': 6, 'U.S. temporarily to allow certain impurities in hand sanitizer': 7, 'U.S. allows impurities in hand sanitizer': 8, 'U.S. considers allowing certain impurities in hand sanitizer': 9, 'U.S. to allow new chemicals in hand sanitizer': 10, 'U.S. temporarily to allow certain harmless impurities in hand sanitizer': 11, 'U.S. temporarily allows large amounts of impurities in hand sanitizer.': 12, 'U.S. authorizes inclusion of impurities in hand sanitizer': 13, 'French

In [ ]:
# BERT tokenize all sentences in the unique sentences, We'll later convert this encoding to hyperbolic
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# tokenized_sentences = tokenizer(unique_sentences_list , padding=True, truncation=True, return_tensors="pt")
# print("First sentence tokenized = ",tokenized_sentences['input_ids'][0], "\nIts attention mask = ",tokenized_sentences['attention_mask'][0])

First sentence tokenized =  tensor([  101,  6504, 10220,  4443,  2000,  8592,  2911,  2007,  2053, 21887,
        23350,  3572,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]) 
Its attention mask =  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
eucledian_embeddings_all = model.encode(unique_sentences_list)
print("embeddings shape = ",eucledian_embeddings_all.shape)

embeddings shape =  (1284, 384)


In [7]:
print("First sentence tokenized = ",eucledian_embeddings_all[0])

First sentence tokenized =  [ 4.86408994e-02  9.85469297e-02  7.82749336e-03  7.68095814e-03
  6.71184063e-02 -6.67509511e-02 -4.61634882e-02 -2.96602491e-04
  4.89992946e-02 -3.66893709e-02  9.93568897e-02 -6.24099523e-02
  7.38892555e-02  7.63967931e-02  8.10542188e-05 -4.78565991e-02
 -2.47218786e-03 -1.00447021e-01  7.60206804e-02  1.02885716e-01
 -6.28650114e-02  7.63936862e-02 -3.01487185e-02 -3.37711275e-02
 -7.53155574e-02  8.82957689e-03  6.46419674e-02 -3.06526874e-03
 -2.83909254e-02  2.70752124e-02 -3.50632295e-02  4.84090112e-02
 -8.03167894e-02  6.10431768e-02  6.68990240e-02 -8.87657888e-03
 -1.15681002e-02 -2.29133777e-02  2.04395652e-02 -1.16177145e-02
  2.10220050e-02 -1.49176093e-02  1.48443297e-01 -2.44956426e-02
  9.14681479e-02 -6.46927357e-02 -3.47170196e-02 -4.78213727e-02
  5.41610532e-02 -2.43215617e-02  1.61878597e-02 -8.06996040e-03
 -4.99835201e-02 -4.07142416e-02 -5.50299026e-02  2.85566859e-02
 -6.99954033e-02 -9.69837513e-03  2.47538611e-02 -5.00480756e-

In [10]:
trees_dict = {}     # create an dict trees in the format {parent:[children]}
neg_trees_dict = {}
trees_list = []     # create an array of tree edges of the format [[parent,children]] like in the .edges files in ParseTreeEmbeddings.ipynb
G = nx.Graph()
for line in lines:
    parent_id = sentence_ids[line[0]]
    child_id = sentence_ids[line[1]]
    trees_list.append([parent_id,child_id])
    if parent_id in trees_dict:
        if child_id not in trees_dict[parent_id]:
            trees_dict[parent_id].append(child_id)
    else:
        trees_dict[parent_id]=[child_id]

for line in neg_lines:
    parent_id = sentence_ids[line[0]]
    child_id = sentence_ids[line[1]]
    if parent_id in neg_trees_dict:
        if child_id not in neg_trees_dict[parent_id]:
            neg_trees_dict[parent_id].append(child_id)
    else:
        neg_trees_dict[parent_id]=[child_id]

G.add_edges_from(trees_list)
print("Positive lines - ")
for line in lines[:3]:
    print(line)

print("Negative lines - ")
for line in neg_lines[:3]:
    print(line)

connected_components = list(nx.connected_components(G))
for i in range(7):
    print(i,unique_sentences_list[i])

print("trees list = ", trees_list)
print("trees dict = ", trees_dict)  #tree_dict and neg_trees_dict is only for plotting the graph. Wont include them finally
print("negtive trees dict = ", neg_trees_dict)
print("Connected components = ", connected_components)
indices = [i for i in range(len(connected_components))]
print("Unique connected components = ", len(indices))

# now ill combine the trees_list data and connected_components data to form an array where each element will be a connected graph
belongs_to_tree = {}
for i,tree in enumerate(connected_components):
    for node in tree:
        belongs_to_tree[node]=i
print("belongs_to_tree = ", belongs_to_tree)

trees = [[] for _ in range(len(connected_components))]
for edge in trees_list:
    trees[belongs_to_tree[edge[0]]].append(edge)

print("trees = ", trees)

euclidean_embeddings = [torch.tensor([eucledian_embeddings_all[i] for i in component]) for component in connected_components]

print(f"euclidean_embeddings[0].shape={euclidean_embeddings[0].shape} euclidean_embeddings[0]={euclidean_embeddings[0]}")

Positive lines - 
['Thailand refuses entry to cruise ship with no coronavirus cases', 'Thailand bans cruise ship even without any coronavirus cases on board.', '1']
['Thailand decides on cruise ship with no coronavirus cases', 'Thailand refuses entry to cruise ship with no coronavirus cases', '1']
['Thailand refuses entry to cruise ship with no coronavirus cases', 'Thailand bans cruise ships with no coronavirus cases', '1']
Negative lines - 
['Thailand refuses entry to cruise ship with no coronavirus cases', 'thailand refuses entry to cruise ships with no cases of coronavirus', '0']
['Thailand decides on cruise ship with no coronavirus cases', 'thailand decides to use a cruise ship with no cases of coronavirus', '0']
['Thailand refuses entry to cruise ship with no coronavirus cases', 'thailand refuses to allow passengers on cruise ships with no cases of coronavirus', '0']
0 Thailand refuses entry to cruise ship with no coronavirus cases
1 Thailand bans cruise ship even without any coro

<ipython-input-10-a8f57d0136c1>:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  euclidean_embeddings = [torch.tensor([eucledian_embeddings_all[i] for i in component]) for component in connected_components]


In [11]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances # for nlp tasks cosine similarity measures closeness better than eucledian. idk chatgpt said so.
# Create a sample tree structure
tree_structure = trees_dict

# Create a directed graph
dot = Digraph(comment='Tree')

# Add nodes and edges to the graph
for parent, children in tree_structure.items():
    dot.node('"{}"'.format(unique_sentences_list[parent]))
    for child in children:
        dot.edge('"{}"'.format(unique_sentences_list[parent]), '"{}"'.format(unique_sentences_list[child]), \
                 label = str(cosine_similarity([eucledian_embeddings_all[parent]], [eucledian_embeddings_all[child]])))
    #     if child in neg_trees_dict:
    #         parent2 = child
    #         children2 = neg_trees_dict[child]
    #         for child2 in children2:
    #             dot.edge('"{}"'.format(unique_sentences_list[parent2]), '"{}"'.format(unique_sentences_list[child2]), \
    #                 label = str(cosine_similarity([tokenized_sentences['input_ids'][parent2]], [tokenized_sentences['input_ids'][child2]])), \
    #                     color="chocolate1")
    # # add unrealted simialr sentences id present
    # if parent in neg_trees_dict:
    #     children2 = neg_trees_dict[parent]
    #     for child2 in children2:
    #         dot.edge('"{}"'.format(unique_sentences_list[parent]), '"{}"'.format(unique_sentences_list[child2]), \
    #              label = str(cosine_similarity([tokenized_sentences['input_ids'][parent]], [tokenized_sentences['input_ids'][child2]])), \
    #                 color="chocolate1")

# Save the graph as a PDF or PNG
dot.render('tree_bert_cosine', format='pdf', cleanup=True)

# Adding connections to next unconnected tree to show the bert encding similarities between some unrealted texts
for i in range(len(connected_components)-1):
    first_tree_node = None
    for component in connected_components[i]:
        if component not in trees_dict:
            first_tree_node = component
            break
    second_tree_node = None
    for component in connected_components[i+1]:
        if component not in trees_dict:
            second_tree_node = component
            break

    dot.node('"{}"'.format(unique_sentences_list[first_tree_node]))
    dot.edge('"{}"'.format(unique_sentences_list[first_tree_node]), '"{}"'.format(unique_sentences_list[second_tree_node]), \
                label = str(cosine_similarity([eucledian_embeddings_all[first_tree_node]], [eucledian_embeddings_all[second_tree_node]])), \
                color="red")


# Save the graph as a PDF or PNG
dot.render('tree_bert_cosine_with_unrealted_connection', format='pdf', cleanup=True)

# Add nodes and edges to the graph from negative matches
# for parent, children in neg_trees_dict.items():
#     dot.node('"{}"'.format(unique_sentences_list[parent]))
#     for child in children[:1]:
#         dot.edge('"{}"'.format(unique_sentences_list[parent]), '"{}"'.format(unique_sentences_list[child]), \
#                  label = str(cosine_  similarity([tokenized_sentences['input_ids'][parent]], [tokenized_sentences['input_ids'][child]])), \
#                  color = "chocolate1")

# # Save the graph as a PDF or PNG
# dot.render('tree_bert_cosine_with_similar_unrealted_connections', format='pdf', cleanup=True)

# Colons are replaced with ", port" and nothing is getting printed in the graph from colon onwards
# Warning: node First step to legalizing marijuana in Switzerland, port  allow legal use of marijuana for research. unrecognized
# Warning: node French Health Ministry confirms, port  COVID-19 cases drop drastically unrecognized
# Warning: node Russian Railways, port  Coronavirus outbreak to impact freight traffic to Europe unrecognized

'tree_bert_cosine_with_unrealted_connection.pdf'

In [12]:
# Code from learning_utils.py

EPS = 1e-15 # for numerical stability
PROJ_EPS = 1e-5
MAX_TANH_ARG = 15.0

# Dot prod
def torch_dot(x, y):
    return torch.sum(x * y, dim=1, keepdim=True)

# L2 norm
def torch_norm(x):
    return torch.norm(x, dim=1, keepdim=True)

# Inverse hyperbolic tangent applied element-wise to a tensor
def t_arctanh(v):
    return 0.5*torch.log((1+v)/(1-v))

# Inverse hyperbolic tangent with clipping
def torch_atanh(x):
    return t_arctanh(torch.min(x, torch.tensor(1. - EPS)))  # just clips values and passes to arctanh which does the actual log(1+v/1-v)

# Hyperbolic tangent with clipping
def torch_tanh(x):
    return torch.tanh(torch.min(torch.max(x, torch.tensor(-MAX_TANH_ARG)), torch.tensor(MAX_TANH_ARG)))

# its fully commented out??
def torch_project_hyp_vec(v, c=1):
    """Projects the hyperbolic vectors to the inside of the ball."""
    # clip_norm = torch.tensor(1-PROJ_EPS)
    # clipped_v = F.normalize(v, p=2, dim=1)*clip_norm
    # return clipped_v
    return v

def torch_hyp_add(u, v, c=1):
    """Accepts torch tensors u, v and returns their sum in hyperbolic
    space in tensor format. Radius of the open ball is 1/sqrt(c). """
    v = v+torch.tensor(EPS)
    torch_dot_u_v = 2 * torch_dot(u, v)
    torch_norm_u_sq = torch_dot(u,u)
    torch_norm_v_sq = torch_dot(v,v)
    denominator = 1. + torch_dot_u_v + torch_norm_v_sq * torch_norm_u_sq
    result = (1. + torch_dot_u_v + torch_norm_v_sq) / denominator * u + (1. - torch_norm_u_sq) / denominator * v
    return torch_project_hyp_vec(result)

def torch_mv_mul_hyp(M, x, c=1):
    x = x + torch.tensor(EPS)
    Mx = torch.matmul(x, M)+torch.tensor(EPS)
    MX_norm = torch_norm(Mx)
    x_norm = torch_norm(x)
    result = torch_tanh(MX_norm / x_norm * torch_atanh(x_norm)) / MX_norm * Mx
    return torch_project_hyp_vec(result, c)

In [13]:
#Hyperbolic modules.

class HypLinear(nn.Module):
    """Applies a hyperbolic "linear" transformation to the incoming data: :math:`y = xA^T + b`
       Uses hyperbolic formulation of addition, scaling and matrix multiplication.
    """

    def __init__(self, in_features, out_features, bias=True):
        super(HypLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))

        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(1, out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input_):
        result = torch_hyp_add(torch_mv_mul_hyp(torch.transpose(self.weight,0,1), input_), self.bias) #(batch, input) x (input, output)
        return result

    def extra_repr(self):
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )

In [14]:
#Riemannian SGD

from torch.optim.optimizer import Optimizer, required
spten_t = torch.sparse.FloatTensor


def poincare_grad(p, d_p):
    """
    Calculates Riemannian grad from Euclidean grad.
    Args:
        p (Tensor): Current point in the ball
        d_p (Tensor): Euclidean gradient at p
    """
    if d_p.is_sparse:
        p_sqnorm = torch.sum(
            p.data[d_p._indices()[0].squeeze()] ** 2, dim=1,
            keepdim=True
        ).expand_as(d_p._values())
        n_vals = d_p._values() * ((1 - p_sqnorm) ** 2) / 4
        d_p = spten_t(d_p._indices(), n_vals, d_p.size())
    else:
        p_sqnorm = torch.sum(p.data ** 2, dim=-1, keepdim=True)
        d_p = d_p * ((1 - p_sqnorm) ** 2 / 4).expand_as(d_p)

    return d_p


def euclidean_grad(p, d_p):
    return d_p


def retraction(p, d_p, lr):
    # Gradient clipping.
    if torch.all(d_p < 1000) and torch.all(d_p>-1000):
        p.data.add_(-lr, d_p)


class RiemannianSGD(Optimizer):
    """Riemannian stochastic gradient descent.
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        rgrad (Function): Function to compute the Riemannian gradient from
            an Euclidean gradient
        retraction (Function): Function to update the parameters via a
            retraction of the Riemannian gradient
        lr (float): learning rate
    """

    def __init__(self, params, lr=required, rgrad=required, retraction=required):
        defaults = dict(lr=lr, rgrad=rgrad, retraction=retraction)
        super(RiemannianSGD, self).__init__(params, defaults)

    def step(self, lr=None):
        """Performs a single optimization step.
        Arguments:
            lr (float, optional): learning rate for the current update.
        """
        loss = None

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if lr is None:
                    lr = group['lr']
                d_p = group['rgrad'](p, d_p)
                group['retraction'](p, d_p, lr)

        return loss

In [23]:
# Distortion calculations

def acosh(x):   # acosh = log(x+sqrt(x^2 - 1))
    return torch.log(x + torch.sqrt(x**2-1))

def dist_h(u,v):
    z  = 2*torch.norm(u-v,2)**2 # formula at - https://en.wikipedia.org/wiki/Poincar%C3%A9_disk_model
    uu = 1. + torch.div(z,((1-torch.norm(u,2)**2)*(1-torch.norm(v,2)**2)))  # we can just just use torch.norm(u) instead of torch.norm(u,2) L2 is by default
    return acosh(uu)

def distance_matrix_hyperbolic(input):
    row_n = input.shape[0]
    dist_mat = torch.zeros(row_n, row_n, device=device) # dist_mat stores the hyperbolic (geodesic) distance between each pair of points
    for row in range(row_n):
        for i in range(row_n):
            if i != row:
                dist_mat[row, i] = dist_h(input[row,:], input[i,:])
    return dist_mat

# test_input = torch.tensor([[1,1,1,1,1,1,1,1],[0,3,1,0,0,3,1,0],[1,0,0,1,1,0,0,1], [2,0,0,2,2,0,0,2]], dtype=torch.float32)
# print("test_input = ", test_input)
# print(distance_matrix_hyperbolic(test_input))

def entry_is_good(h, h_rec): return (not torch.isnan(h_rec)) and (not torch.isinf(h_rec)) and h_rec != 0 and h != 0

def distortion_entry(h,h_rec):
    avg = abs(h_rec - h)/h
    return avg

def distortion_row(H1, H2, n, row):
    avg, good = 0, 0
    for i in range(n):
        if i != row and entry_is_good(H1[i], H2[i]):    # if H2[i] is inf or nan or 0 then there is a problem in distortion_entry(). SO entry_is_good() is checked
            _avg = distortion_entry(H1[i], H2[i])
            good += 1
            avg  += _avg
    if good > 0:
        avg /= good
    else:
        avg, good = torch.tensor(0., device=device, requires_grad=True), torch.tensor(0., device=device, requires_grad=True)
    return (avg, good)

def distortion(H1, H2, n, jobs=16):
#     dists = Parallel(n_jobs=jobs)(delayed(distortion_row)(H1[i,:],H2[i,:],n,i) for i in range(n))
    dists = (distortion_row(H1[i,:],H2[i,:],n,i) for i in range(n))
    to_stack = [tup[0] for tup in dists]
    avg = torch.stack(to_stack).sum()/n
    return avg
# Does Euclidean to hyperbolic mapping using series of FC layers.
# We use ground truth distance matrix for the pair since the distortion for hyperbolic embs are really low.
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def get_dist_mat(G,node_to_ids):
    lengths = dict(nx.all_pairs_dijkstra_path_length(G))
    # print("lengths = ", lengths)
    n = nx.number_of_nodes(G)
    dist_mat = np.full((n, n), np.inf)

    for i, (node, lengths_to_node) in enumerate(lengths.items()):
        for target_node, length in lengths_to_node.items():
            dist_mat[node_to_ids[node], node_to_ids[target_node]] = length

    return dist_mat


def pairfromidx(idx):
    # input_tensor = tensorFromSentence(input_vocab, filtered_sentences[idx])
    input_sentence_ids = connected_components[idx]
    input_tensor = torch.tensor(list(input_sentence_ids)).view(-1, 1)
    # G = load_graph("random_trees/"+str(idx)+".edges")
    G = nx.from_edgelist(trees[idx])
    node_to_ids = {}
    i=0
    for node in input_sentence_ids:
        node_to_ids[node]=i
        i=i+1
    target_matrix = get_dist_mat(G,node_to_ids)
    target_tensor = torch.from_numpy(target_matrix).float().to(device)
    target_tensor.requires_grad = False
    n = G.order()
    # print("input_tensor = ", input_tensor)
    # print("target_tensor = ", target_tensor)
    # print("Order of graph = ", n)
    return (input_tensor, target_tensor, n) # sentences_text[idx] was also returned but idk if its required

    # the word pairfromidx created distance matrix for sentences.
    # for our sentence case -> we will give ids to each seperate tree and run pairfromidx for that tree.
    # well return the

def trainFCHyp(input_matrix, ground_truth, n, mapping, mapping_optimizer, max_length=50):
    mapping_optimizer.zero_grad()

    loss = 0
    # print("input_matrix = ", input_matrix)
    output = mapping(input_matrix.float())
    dist_recovered = distance_matrix_hyperbolic(output)
    loss += distortion(ground_truth, dist_recovered, n)
    loss.backward()
    mapping_optimizer.step()

    return loss.item()


def trainFCIters(mapping, n_epochs=5, n_iters=500, print_every=50, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0

    mapping_optimizer = RiemannianSGD(mapping.parameters(), lr=learning_rate, rgrad=poincare_grad, retraction=retraction)
    # training_pairs = [pairfromidx(idx) for idx in range(n_iters)]
    training_pairs = [pairfromidx(idx) for idx in range(len(connected_components))]
    for i in range(n_epochs):
        print("Starting epoch "+str(i))
        iter=1
        for idx in indices:
            input_matrix = euclidean_embeddings[idx]
            target_matrix = training_pairs[idx][1]
            n = training_pairs[idx][2]
            loss = trainFCHyp(input_matrix, target_matrix, n, mapping, mapping_optimizer)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, print_loss_avg))

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

            iter+=1

input_size = 384
output_size = 2
mapping = nn.Sequential(
          nn.Linear(input_size, 50).to(device),
          nn.ReLU().to(device),
          nn.Linear(50, output_size).to(device),
          nn.ReLU().to(device))
print("trees list = ", trees_list)
print("Connected components = ", connected_components)
print("Unique connected components = ", len(connected_components))
print("All trees = ", trees)
print("total trees = ", len(trees ))
trainFCIters(mapping)

trees list =  [[0, 1], [2, 0], [0, 3], [4, 0], [0, 5], [0, 6], [7, 8], [9, 7], [7, 10], [11, 7], [7, 12], [7, 13], [14, 15], [16, 14], [14, 17], [18, 14], [14, 19], [14, 20], [21, 22], [23, 21], [21, 24], [25, 21], [21, 26], [21, 27], [28, 29], [30, 28], [28, 31], [32, 28], [28, 33], [28, 34], [35, 36], [37, 35], [35, 38], [39, 35], [35, 40], [35, 41], [42, 43], [44, 42], [42, 45], [42, 46], [42, 47], [48, 49], [49, 50], [49, 51], [49, 52], [49, 53], [54, 55], [56, 57], [56, 58], [59, 56], [56, 60], [56, 61], [62, 63], [64, 62], [62, 65], [62, 66], [67, 68], [69, 67], [67, 70], [71, 67], [67, 72], [67, 73], [74, 75], [76, 74], [74, 77], [78, 74], [74, 79], [74, 80], [81, 82], [83, 81], [81, 84], [85, 81], [81, 86], [81, 87], [88, 89], [90, 88], [88, 91], [92, 88], [88, 93], [88, 94], [95, 96], [97, 95], [95, 98], [99, 95], [95, 100], [95, 101], [102, 103], [104, 102], [102, 105], [106, 102], [102, 107], [102, 108], [109, 110], [111, 109], [109, 112], [113, 109], [109, 114], [109, 115],

In [24]:
new_embeddings = mapping(torch.tensor(eucledian_embeddings_all))
new_embeddings

tensor([[0.0190, 0.0271],
        [0.0237, 0.0339],
        [0.0237, 0.0238],
        ...,
        [0.0338, 0.0088],
        [0.0471, 0.0347],
        [0.0440, 0.0402]], grad_fn=<ReluBackward0>)